In [1]:
%store -r __importData
import pandas as pd
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#from nltk.tokenize import word_tokenize
import jieba
jieba.set_dictionary('dict.txt.big')

C:\Users\S\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [38]:
__importData

import group_table


C:\Users\S\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


import airline_table


C:\Users\S\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


import order_table
import train_set
import test_set


In [70]:
def get_doc_vec(x):
    prod_vec = pd.DataFrame(columns=["group_id"] + ["doc_vec_" + str(i + 1) for i in range(20)],
                            index=range(1))
    vec = model.infer_vector(list(jieba.cut(corpus[x["product_name"]])))
    prod_vec["group_id"] = x["group_id"]
    for col, v in zip(prod_vec.columns[1:], vec):
        prod_vec[col] = v
    return prod_vec

In [23]:
corpus = list(group_table.product_name.fillna("NaN").unique())
tagged_corpus = [TaggedDocument(words=list(jieba.cut(_d)), tags=[str(i)]) for i, _d in enumerate(corpus)]

max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm=0)
  
model.build_vocab(tagged_corpus)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_corpus,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

C:\Users\S\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


C:\Users\S\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [24]:
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)

[('4454', 0.8647611737251282), ('50', 0.830884575843811), ('4478', 0.8222824335098267), ('16', 0.8218321800231934), ('81', 0.8111234903335571), ('32', 0.809941291809082), ('4466', 0.7920123934745789), ('51', 0.7919424772262573), ('4460', 0.7898001670837402), ('12', 0.7891075611114502)]


C:\Users\S\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [48]:
model.infer_vector(list(jieba.cut(corpus[0])))[0]

-0.078309886

In [ ]:
group_table.product_name.fillna("NaN", inplace=True)
prod_map = {}
corpus = list(group_table.product_name.unique())
for i, cp in enumerate(corpus):
    prod_map[cp] = i
group_table["product_name"] = group_table.product_name.map(prod_map)
prod_vec = pd.DataFrame(columns=["product_name"] + ["doc_vec_" + str(i + 1) for i in range(20)],
                            index=range(len(corpus)))
for i in range(len(corpus)):
    prod_vec.iloc[i] = [i] + list(model.docvecs[i])
group_table = pd.merge(group_table, prod_vec, on="product_name", how="left")